# Модуль re

Основные функции модуля re:

match() - ищет последовательность в начале строки \
search() - ищет первое совпадение с шаблоном\
findall() - ищет все совпадения с шаблоном. Возвращает результирующие строки в виде списка\
finditer() - ищет все совпадения с шаблоном. Возвращает итератор\
compile() - компилирует регулярное выражение. К этому объекту затем можно применять все перечисленные функции\

### *Функция Match*

In [2]:
import re
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'
match = re.search(r'Host (\S+) in vlan (\d+) .* port (\S+) and port (\S+)', log)
match


<_sre.SRE_Match object; span=(47, 124), match='Host f03a.b216.7ad7 in vlan 10 is flapping betwee>

Вывод в 3 строке просто отображает информацию об объекте. Поэтому не стоит полагаться на то, что отображается в части match, так как отображаемая строка обрезается по фиксированному количеству знаков.

### Функция search()




* используется для поиска подстроки, которая соответствует шаблону
* возвращает объект Match, если подстрока найдена
* возвращает None, если подстрока не найдена
Функция search подходит в том случае, когда надо найти только одно совпадение в строке, например, когда регулярное выражение описывает всю строку или часть строки.

Рассмотрим пример использования функции search в разборе лог-файла.

В файле log.txt находятся лог-сообщения с информацией о том, что один и тот же MAC слишком быстро переучивается то на одном, то на другом интерфейсе. Одна из причин таких сообщений - петля в сети.

Содержимое файла log.txt:

%SW_MATM-4-MACFLAP_NOTIF: Host 01e2.4c18.0156 in vlan 10 is flapping between port Gi0/16 and port Gi0/24 \
%SW_MATM-4-MACFLAP_NOTIF: Host 01e2.4c18.0156 in vlan 10 is flapping between port Gi0/16 and port Gi0/24  \
%SW_MATM-4-MACFLAP_NOTIF: Host 01e2.4c18.0156 in vlan 10 is flapping between port Gi0/24 and port Gi0/19  \
%SW_MATM-4-MACFLAP_NOTIF: Host 01e2.4c18.0156 in vlan 10 is flapping between port Gi0/24 and port Gi0/16

При этом MAC-адрес может прыгать между несколькими портами. В таком случае очень важно знать, с каких портов прилетает MAC.

Попробуем вычислить, между какими портами и в каком VLAN образовалась проблема. Проверка регулярного выражения с одной строкой из log-файла:

In [6]:
import re
log = '%SW_MATM-4-MACFLAP_NOTIF: Host 01e2.4c18.0156 in vlan 10 is flapping between port Gi0/16 and port Gi0/24'
match = re.search(r'Host \S+ 'r'in vlan (\d+) '
                  r'is flapping between port '
                  r'(\S+) and port (\S+)', log)
match

<_sre.SRE_Match object; span=(26, 104), match='Host 01e2.4c18.0156 in vlan 10 is flapping betwee>

### Функция findall()

Функция findall():

* используется для поиска всех непересекающихся совпадений в шаблоне
возвращает:
  * список строк, которые описаны регулярным выражением, если в регулярном выражении нет групп
  * список строк, которые совпали с регулярным выражением в группе, если в регулярном выражении одна группа
  * список кортежей, в которых находятся строки, которые совпали с выражением в группе, если групп несколько
  
Рассмотрим работу findall на примере вывода команды sh mac address-table:

In [7]:
mac_address_table = open('CAM_table.txt').read()
print(mac_address_table)

          Mac Address Table
-------------------------------------------

Vlan    Mac Address       Type        Ports
----    -----------       --------    -----
 100    a1b2.ac10.7000    DYNAMIC     Gi0/1
 200    a0d4.cb20.7000    DYNAMIC     Gi0/2
 300    acb4.cd30.7000    DYNAMIC     Gi0/3
 100    a2bb.ec40.7000    DYNAMIC     Gi0/4
 500    aa4b.c550.7000    DYNAMIC     Gi0/5
 200    a1bb.1c60.7000    DYNAMIC     Gi0/6
 300    aa0b.cc70.7000    DYNAMIC     Gi0/7


Первый пример - регулярное выражение без групп. В этом случае findall возвращает список строк, которые совпали с регулярным выражением.

Например, с помощью findall можно получить список строк с соответствиями vlan - mac - interface и избавиться от заголовка в выводе команды:

In [8]:
import re
re.findall(r'\d+ +\S+ +\w+ +\S+', mac_address_table)

['100    a1b2.ac10.7000    DYNAMIC     Gi0/1',
 '200    a0d4.cb20.7000    DYNAMIC     Gi0/2',
 '300    acb4.cd30.7000    DYNAMIC     Gi0/3',
 '100    a2bb.ec40.7000    DYNAMIC     Gi0/4',
 '500    aa4b.c550.7000    DYNAMIC     Gi0/5',
 '200    a1bb.1c60.7000    DYNAMIC     Gi0/6',
 '300    aa0b.cc70.7000    DYNAMIC     Gi0/7']

Обратите внимание, что findall возвращает список строк, а не объект Match.

Как только в регулярном выражении появляется группа, findall ведет себя по-другому. Если в выражении используется одна группа, findall возвращает список строк, которые совпали с выражением в группе:

In [9]:
re.findall(r'\d+ +(\S+) +\w+ +\S+', mac_address_table)

['a1b2.ac10.7000',
 'a0d4.cb20.7000',
 'acb4.cd30.7000',
 'a2bb.ec40.7000',
 'aa4b.c550.7000',
 'a1bb.1c60.7000',
 'aa0b.cc70.7000']

### Функция finditer

Функция finditer():

* используется для поиска всех непересекающихся совпадений в шаблоне
* возвращает итератор с объектами Match
* finditer возвращает итератор даже в том случае, когда совпадение не найдено
Функция finditer отлично подходит для обработки тех команд, вывод которых отображается столбцами. Например, sh ip int br, sh mac address-table и др. В этом случае его можно применять ко всему выводу команды.

Пример вывода sh ip int br:

In [14]:
sh_ip_int_br = '''
   ...: R1#show ip interface brief
   ...: Interface             IP-Address      OK? Method Status           Protocol
   ...: FastEthernet0/0       15.0.15.1       YES manual up               up
   ...: FastEthernet0/1       10.0.12.1       YES manual up               up
   ...: FastEthernet0/2       10.0.13.1       YES manual up               up
   ...: FastEthernet0/3       unassigned      YES unset  up               up
   ...: Loopback0             10.1.1.1        YES manual up               up
   ...: Loopback100           100.0.0.1       YES manual up               up
   ...: '''
result = re.finditer(r'(\S+) +'
                  r'([\d.]+) +'
                  r'\w+ +\w+ +'
                  r'(up|down|administratively down) +'
                  r'(up|down)',
                  sh_ip_int_br)
result

### Функция compile

В Python есть возможность заранее скомпилировать регулярное выражение, а затем использовать его. Это особенно полезно в тех случаях, когда регулярное выражение много используется в скрипте.

Использование компилированного выражения может ускорить обработку, и, как правило, такой вариант удобней использовать, так как в программе разделяется создание регулярного выражения и его использование. Кроме того, при использовании функции re.compile создается объект RegexObject, у которого есть несколько дополнительных возможностей, которых нет в объекте MatchObject.

Для компиляции регулярного выражения используется функция re.compile:

In [ ]:
regex = re.compile(r'\d+ +\S+ +\w+ +\S+')

У объекта RegexObject доступны такие методы и атрибуты:

In [ ]:
[method for method in dir(regex) if not method.startswith('_')]

# Модуль difflib

Этот модуль предоставляет классы и функции для сравнения последовательностей. Он может быть использован для сравнения файлов, и может создавать разностную информацию в различных форматах, включая HTML, контекста и унифицированные diff’ы.

### class difflib.SequenceMatcher
Это гибкий класс для сравнения пар последовательностей любого типа при условии, что элементы последовательности являются хэшируемым. Базовый алгоритм предшествует, и является немного более модным, чем алгоритм, опубликованный в конце 1980-х годов Ratcliff и Obershelp под гиперболическим названием «сопоставление гештальт-паттернов». Идея состоит в том, чтобы найти самую длинную смежную подпоследовательность, которая не содержит «нежелательных» элементов; эти «нежелательные» элементы являются неинтересными в некотором смысле, например пустые строки или пробелы. (Обработка нежелательной почты является расширением алгоритма Ratcliff и Obershelp.) Эта же идея затем рекурсивно применяется к частям последовательностей слева и справа от совпадающей подпоследовательности. Это не дает минимального редактирования последовательности, но как правило, возвращают совпадения, которые «выглядят правильно» для людей.

Тайминг: базовым алгоритмом Ratcliff-Obershelp работает за кубическое время в наихудшем случае и квадратичное время в ожидаемом случае. SequenceMatcher работает за квадратичное время для наихудшего случая и поведение ожидаемого случая, зависит от сложного способа определения количества элементов содержащихся в последовательности; лучшее время случая линейно.

Автоматическая мусорная эвристика: SequenceMatcher поддерживает эвристику, которая автоматически рассматривает определенные элементы последовательности как нежелательные. Эвристика подсчитывает, сколько раз каждый отдельный элемент появляется в последовательности. Если дубликаты элемента (после первого) составляют более 1% последовательности, а длина последовательности составляет не менее 200 элементов, этот элемент помечается как «популярный» и рассматривается как нежелательный для целей сопоставления последовательности. Этот эвристический параметр можно отключить, установив для аргумента autojunk значение False при создании SequenceMatcher.

### Примеры SequenceMatcher

#### ratio()
Возвращает меру сходства последовательностей в виде float в диапазоне [0, 1].

Где T - общее число элементов в обеих последовательностях, а M - количество совпадений, это 2,0* M/T. Обратите внимание, что 1.0, если последовательности идентичны, и 0.0, если у них нет ничего общего.

Этот метод затратен в вычислениях, если get_matching_blocks() или get_opcodes() еще не былы вызван, в этом случае вы можете попытаться quick_ratio() или real_quick_ratio() сначала получить верхнюю границу.

В этом примере сравниваются две строки, рассматривая пробелы как «мусор»:

In [21]:
import difflib
s = difflib.SequenceMatcher(lambda x: x == " ",
                    "private Thread currentThread;",
                    "private volatile Thread currentThread;")
print(round(s.ratio(), 3))

0.866


ratio() возвращает float в [0, 1], измеряя подобие последовательностей. Как правило, ratio() значение более 0.6 означает, что последовательности являются близкими совпадениями.

####find_longest_match(alo, ahi, blo, bhi)
Найти самый длинный соответствующий блок в a[alo:ahi] и b[blo:bhi].

Если isjunk пропущен или None, find_longest_match() возвращает (i, j, k) так, что a[i:i+k] равен b[j:j+k], где alo <= i <= i+k <= ahi и blo <= j <= j+k <= bhi. Для всех (i', j', k'), удовлетворяющих этим условиям, также выполняются дополнительные условия k >= k', i <= i' и если i == i', j <= j'. Другими словами, из всех максимальных совпадающих блоков, возвращает тот, который начинается раньше всего в a и из всех максимальных совпадающих блоков, которые начинаются с самого раннего в a, возвращает тот, который начинается раньше всего в b.

In [23]:
import difflib
s = difflib.SequenceMatcher(None, " abcd", "abcd abcd")
s.find_longest_match(0, 5, 0, 9)


Match(a=0, b=4, size=5)

###class difflib.Differ(linejunk=None, charjunk=None)
Дополнительные ключевые параметры linejunk и charjunk для функций фильтра (или None):

linejunk: функция, принимающая один строковый аргумент и возвращающая значение true, если строка является нежелательной. Значение по умолчанию равно None, что означает, что ни одна строка не считается нежелательной.

charjunk: функция, принимающая один символ (строка длиной 1) и возвращающая значение true, если символ является нежелательным. Значение по умолчанию равно None, что означает, что ни один символ не считается нежелательным.

Эти функции нежелательной фильтрации ускоряют сопоставление для поиска различий и не приводят к игнорированию различных строк или символов. Прочитайте описание параметра метода find_longest_match() isjunk для объяснения.

Differ объекты используются (создаются дельты) одним методом:

* compare(a, b)
Сравнить две последовательности строк и создать дельту (последовательность строк).

Каждая последовательность должна содержать отдельную одну строку, заканчивающуюся символом новой строки. Такие последовательности могут быть получены из метода readlines() файлоподобных объектов. Сформированная дельта также состоит из завершенных симолом новой строки строк, готовых к печати как есть с помощью метода writelines() файлоподобного объекта.

#### Пример

В этом примере сравниваются два текста. Сначала мы настраиваем тексты, последовательностью отдельных однострочных строк, заканчивающихся символом новой строки (такие последовательности также могут быть получены из метода readlines() файлоподобных объектов):

In [26]:
from difflib import Differ
from pprint import pprint

text1 = '''  1. Beautiful is better than ugly.
  2. Explicit is better than implicit.
  3. Simple is better than complex.
  4. Complex is better than complicated.
'''.splitlines(keepends=True)
len(text1)

text1[0][-1]

text2 = '''  1. Beautiful is better than ugly.
  3.   Simple is better than complex.
  4. Complicated is better than complex.
  5. Flat is better than nested.
'''.splitlines(keepends=True)
d = Differ()
result = list(d.compare(text1, text2))
pprint(result)

['    1. Beautiful is better than ugly.\n',
 '-   2. Explicit is better than implicit.\n',
 '-   3. Simple is better than complex.\n',
 '+   3.   Simple is better than complex.\n',
 '?     ++\n',
 '-   4. Complex is better than complicated.\n',
 '?            ^                     ---- ^\n',
 '+   4. Complicated is better than complex.\n',
 '?           ++++ ^                      ^\n',
 '+   5. Flat is better than nested.\n']


### Задание 

Найдите любой небольшой текст и с помощью re найдите предложения, которые начинаются с согласной буквы, а заканчиваются гласной. Далее с помощью difflib найдите разницу между самым большим и самым маленьким предложением. 